# GrainPalette – Rice Type Classification Using Deep Learning
## A Deep Learning Odyssey using MobileNetV2 and Transfer Learning

### Introduction
This project classifies five types of rice grains using transfer learning with MobileNetV2.
It demonstrates the power of CNNs in agricultural image classification using simulated image data.

In [ ]:
# Import required libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from PIL import Image

### Simulate a dummy rice dataset (for demo only)
We generate synthetic image data since no real dataset is included.

In [ ]:
rice_types = ['Basmati', 'Jasmine', 'Arborio', 'Sona Masoori', 'Red Rice']

def generate_dummy_data(samples_per_class=100):
    X, y = [], []
    for i, label in enumerate(rice_types):
        for _ in range(samples_per_class):
            # Dummy image: random pixels
            img = np.random.rand(224, 224, 3)
            X.append(img)
            y.append(i)
    return np.array(X), to_categorical(y, num_classes=len(rice_types))

X, y = generate_dummy_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### Build the Rice Classifier Model with MobileNetV2

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(len(rice_types), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Train the model (demo training)

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=32)

# Optional: plot training accuracy and loss
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.title('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Loss')
plt.legend()
plt.show()

### Upload an image and get prediction

In [ ]:
try:
    from google.colab import files
    uploaded = files.upload()

    for file_name in uploaded.keys():
        img = Image.open(file_name).resize((224, 224)).convert('RGB')
        img_array = np.expand_dims(np.array(img) / 255.0, axis=0)
        prediction = model.predict(img_array)[0]
        predicted_index = np.argmax(prediction)
        confidence = round(100 * prediction[predicted_index], 2)
        print(f"Predicted: {rice_types[predicted_index]} ({confidence}%)")
        plt.imshow(img)
        plt.title(f"{rice_types[predicted_index]} ({confidence}%)")
        plt.axis('off')
        plt.show()
except ImportError:
    print("Not running in Colab. Please place test images in the working directory.")

### Save the model

In [ ]:
model.save('mobilenet_rice_model.h5')